<a href="https://colab.research.google.com/github/Kelseyyyyy/Movie-recommendation-system/blob/main/MY_SCHOOL_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

#sample movie data

movies_dict = {
    'movieId': [1, 2, 3, 4, 5],
    'title': ['Toy Story', 'Jumanji', 'Grumpier Old Men', 'Waiting to Exhale', 'Father of the Bride Part II'],
    'genres': ['Adventure|Animation|Children|Comedy|Fantasy',
               'Adventure|Children|Fantasy',
               'Comedy|Romance',
               'Comedy|Drama|Romance',
               'Comedy']
}

movies = pd.DataFrame(movies_dict)
print(movies)

#sample rating data

ratings_dict = {
    'userId': [1, 1, 1, 2, 2, 2, 3, 3, 4, 5],
    'movieId': [1, 2, 3, 2, 3, 4, 1, 5, 4, 5],
    'rating': [4.0, 4.5, 5.0, 3.0, 4.0, 4.5, 2.0, 3.5, 5.0, 3.0]
}
ratings = pd.DataFrame(ratings_dict)
#preview
print("🎬 Movies:\n", movies)
print("\n⭐ Ratings:\n", ratings)

#creating a user item matrix
user_movie_ratings = ratings.pivot_table(index='userId', columns='movieId', values='rating')
print("\nUser-Movie Rating Matrix:\n", user_movie_ratings)

#fill NaNs
user_movie_ratings.fillna(0, inplace=True)

#checking the matrix
user_movie_ratings

from sklearn.metrics.pairwise import cosine_similarity

# Convert to DataFrame for readability
user_similarity = cosine_similarity(user_movie_ratings)
user_similarity_df = pd.DataFrame(user_similarity, index=user_movie_ratings.index, columns=user_movie_ratings.index)


#showing similarity betweee users
user_similarity_df

def recommend_movies(user_id, user_movie_ratings, user_similarity_df, movies, ratings, top_n=3):
#similar score for user
    sim_scores = user_similarity_df[user_id]
    similar_users = sim_scores.sort_values(ascending=False).index
    user_movies = user_movie_ratings.loc[user_id]
    user_watched = user_movies[user_movies > 0].index.tolist()

    movie_scores = {}
    for sim_user in similar_users:
        sim_user_ratings = user_movie_ratings.loc[sim_user]
        for movie_id, rating in sim_user_ratings.items():
           if movie_id not in user_watched and rating > 0:
              if movie_id not in movie_scores:
                 movie_scores[movie_id] = 0
              movie_scores[movie_id] += rating * sim_scores[sim_user]


#sorting movie scores
    recommended_movies = sorted(movie_scores.items(), key=lambda x: x[1], reverse=True)

#top N recommendations
    recommended_ids = [movie_id for movie_id, score in recommended_movies[:top_n]]
    recommended_titles = movies[movies['movieId'].isin(recommended_ids)]['title'].tolist()

    return recommended_titles
#recommend for user

recommendations = recommend_movies(1, user_movie_ratings, user_similarity_df, movies, ratings)
print("🎥 Recommended Movies for User 1:", recommendations)





   movieId                        title  \
0        1                    Toy Story   
1        2                      Jumanji   
2        3             Grumpier Old Men   
3        4            Waiting to Exhale   
4        5  Father of the Bride Part II   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
🎬 Movies:
    movieId                        title  \
0        1                    Toy Story   
1        2                      Jumanji   
2        3             Grumpier Old Men   
3        4            Waiting to Exhale   
4        5  Father of the Bride Part II   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                             